# Pricing Bermuda option using Monte-carlo regression 

**Objective**: Explore different regression models in Least-square Monte-Carlo algorithm for american style option pricing

For american option paying $g(\tau)$ if exercised at time $\tau<T$, the risk neutral price is given by the optimal stopping problem (for complete market):
$$C(t) = \sup_{t<\tau<T}E_t^Q[e^{-r(\tau - t)}g(\tau)]$$
And verify the following dynamic programming principle:
$$C(t) = max\big[g(t), E_t^Q[e^{-rdt}g(t+dt)]\big]$$ 

The algorithme of least square Monte-Carlo (Longstaff Schwartz) used here is a numerical scheme that solve this problem by fitting, backward recursively in time, regression models that predict the continuation value (value of option at next step previously calculated) given the current state variables (prices of assets). See references for details.

We implement the Bermuda option case which is an approx. of American option for finite number of exercise periods

In [2]:
# pip install git+git://github.com/khrapovs/fangoosterlee
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
%matplotlib inline
from pricing_utils import payoff_put_basket, payoff_worstof_put_basket, LSMC_2D_pricer, GBM_multi
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

def payoff_basket_p(S_T, K):
    a = 1 / S_T.shape[0] * np.ones(S_T.shape[0])
    return payoff_put_basket(a, S_T, K)

def payoff_basket_c(S_T, K):
    a = 1 / S_T.shape[0] * np.ones(S_T.shape[0])
    return payoff_call_basket(a, S_T, K)


## Classical Least-square implementation

In [3]:
N=10000
T=1
M=10
S_0 = np.array([11, 12])
K=20
r=0.05
Rho = np.array([[1,0.2], [0.2,1]]) 
sigma = np.array([0.4, 0.3])
Gamma = np.diag(sigma) * Rho * np.diag(sigma)
Sigma = np.sqrt(Gamma)
R=4

res, npv, P = LSMC_2D_pricer(N, R, T, M, S_0, K, r, sigma,
                               Sigma, payoff_fun=payoff_worstof_put_basket,
                               basis='laguerre')
print('price:', res[0])
print('std:', res[1])
print('err:', res[2])

price: 6.16408014646138
std: 1.605974390445971e-27
err: 4.007461029686965e-16


## Other models for regression 

In [4]:
def LS_backward(P, payoff_fun, K, model,
                poly_features=True, R=6,
                discount_factor=1):
    """
    Learning phase of the Least-square MC algo with model specification, Here the same model is trained 
    multiple time (at each time step)
    Parameters
    ----------
    P (array-like) : multi-dimensional Prices paths array (d x N x T)
    payoff_fun (array-like): payoff fun used to
    model: regressor model suitable for multiple training 
    poly_features (bool): weather to generate polynomial features
    K (float): strike value
    R (int) : max order of polynomial basis
    discount_factor (float): discount factor

    Returns
    -------

    """
    N = P.shape[1] # nb of path
    M = P.shape[2] # nb of time step
    V = payoff_fun(P[:,:,M-1], K)
    if poly_features:
        model = make_pipeline(PolynomialFeatures(degree=R), model)
            
    for t in range(M - 1, -1, -1):
        model.fit(P[:,:,t].T, V)  # use initial_epoch arg if training
                                # Keras MLP (resume training at precedent step)
        cont_val = model.predict(P[:,:,t].T)
        cont_val = cont_val.ravel()
        V = np.where(cont_val < payoff_fun(P[:,:,t], K),
                     payoff_fun(P[:,:,t], K),
                     discount_factor * V)
        
    mean = np.sum(V)/N
    var = np.sum(V-mean)**2/N
    err = np.sqrt(var/N)
    
    return mean, var, err

def LSMC_pricer(N, R, T, M, S_0, K, r, sigma, Sigma, payoff_fun, model, poly_features):
    P_train = GBM_multi(N, T, M, S_0, r, sigma, Sigma)
    delta_t = T/M
    discount_factor = np.exp(-r * delta_t)
    V_mean, V_var, V_err = LS_backward(P_train, payoff_fun, K, model=model, R=R,
                                       discount_factor=discount_factor,
                                       poly_features=poly_features)

    return V_mean, V_var, V_err

### Polynomial regression

In [5]:
from sklearn.linear_model import Ridge

N=100000
T=1
M=10
S_0 = np.array([100])
K=110
r=0.1
Rho = 1
sigma = np.array([0.25])
Gamma = np.diag(sigma) * Rho * np.diag(sigma)
Sigma = np.sqrt(Gamma)
R=3

model = Ridge(alpha=0., solver='auto')

starttime = time.time()
p, _, _ = LSMC_pricer(N, R, T, M, S_0, K, r, sigma, Sigma,
                      payoff_fun=payoff_basket_p, poly_features=True,
                      model=model)
endtime = time.time()
print('\nTime Cost predict:', endtime - starttime, 'seconds')
print('price: ', p)


Time Cost predict: 0.602353572845459 seconds
price:  11.677944378328016


### ANN Regression

In [6]:
from ml_models import MLPRegressor, MLPRegressorPCA
from sklearn.preprocessing import StandardScaler


model = MLPRegressor(
    scaler=StandardScaler(),
    n_feature=4, # two assets and 4 degree basis
    epochs=5,
    batch_size=512,
    loss="mse",
    verbose=0
)

params = {'activation': 'tanh',
         'batch_normalisation': False,
         'beta_1': 0.6,
         'beta_2': 0.4,
         'dropout': 0.,
         'hidden_size': 100,
         'hidden_size_2':50,
         'kernel_initializer': 'normal',
         'learning_rate': 0.00015
         }
model.set_params(**params)

starttime = time.time()
p, _, _ = LSMC_pricer(N, R, T, M, S_0, K, r, sigma, Sigma,
                      payoff_fun=payoff_basket_p, poly_features=True,
                      model=model)
endtime = time.time()
print('\nTime Cost predict:', endtime - starttime, 'seconds')
print('price: ', p)


Time Cost predict: 91.50403833389282 seconds
price:  10.0


In [7]:
N=100000
T=1
M=20
S_0 = np.ones(2) * 100
K=100
r=0.05
Rho = np.array([[1,0.2], [0.2,1]]) 
sigma = np.ones(2) * 0.2
Gamma = np.diag(sigma) * Rho * np.diag(sigma)
Sigma = np.sqrt(Gamma)
R=3

model = MLPRegressor(
    scaler=StandardScaler(),
    n_feature=10, # two assets and 4 degree basis
    epochs=5,
    batch_size=512,
    loss="mse",
    verbose=0
)

starttime = time.time()
p, _, _ = LSMC_pricer(N, R, T, M, S_0, K, r, sigma, Sigma,
                      payoff_fun=payoff_basket_p, poly_features=True,
                      model=model)
endtime = time.time()
print('\nTime Cost predict:', endtime - starttime, 'seconds')
print('price: ', p)


Time Cost predict: 155.146546125412 seconds
price:  3.834202018732696
